In [1]:
import os 
os.chdir('../src/')
from traffic_analysis.d05_evaluation.chunk_evaluator import ChunkEvaluator
from traffic_analysis.d00_utils.data_loader_s3 import DataLoaderS3
from traffic_analysis.d00_utils.load_confs import load_parameters, load_credentials, load_paths

params = load_parameters()
creds = load_credentials()
paths = load_paths()

s3_credentials = creds[paths['s3_creds']]

In [2]:
paths['bucket_name']

'air-pollution-uk'

In [3]:
dl = DataLoaderS3(s3_credentials = s3_credentials, 
                 bucket_name = paths['bucket_name'])

In [4]:
dl.read_json(file_path = "ref/video_names/tims_cameras_all.json")


['raw/videos/2019-07-17/20190717-000416_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-000531_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-000546_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-001647_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-001702_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-001803_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-001818_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-001918_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-001933_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-002032_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-002048_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-002145_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-002201_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-002300_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-002317_00001.03604.mp4',
 'raw/videos/2019-07-17/20190717-002414_00001.02262.mp4',
 'raw/videos/2019-07-17/20190717-002429_00001.03604.mp4',
 'raw/videos/2

In [5]:
dl.list_objects(prefix = 'ref/annotations/cvat/')

['ref/annotations/cvat/10_2019-06-20_06-00-46_00001.01252.xml',
 'ref/annotations/cvat/10_2019-06-28_13-02-53.511426_00001.04534.xml',
 'ref/annotations/cvat/10_2019-06-29_13-01-46.107715_00001.07591.xml',
 'ref/annotations/cvat/11_2019-06-20_05-58-17_00001.01252.xml',
 'ref/annotations/cvat/11_2019-06-20_08-58-01_00001.03601.xml',
 'ref/annotations/cvat/11_2019-06-29_13-01-45.674007_00001.03604.xml',
 'ref/annotations/cvat/12_2019-06-20_06-00-23_00001.06590.xml',
 'ref/annotations/cvat/12_2019-06-29_13-01-40.228706_00001.04336.xml',
 'ref/annotations/cvat/13_2019-06-20_06-00-40_00001.04534.xml',
 'ref/annotations/cvat/13_2019-06-29_13-01-21.257815_00001.09560.xml',
 'ref/annotations/cvat/14_2019-06-20_06-00-18_00001.04280.xml',
 'ref/annotations/cvat/14_2019-06-29_13-01-19.744908_00001.05900.xml',
 'ref/annotations/cvat/15_2019-06-29_13-01-03.094068_00001.01252.xml',
 'ref/annotations/cvat/15_2019-07-02_02-58-22.337290_00001.07382.xml',
 'ref/annotations/cvat/17_2019-07-01_03-01-51.37

In [14]:
result = dl.client.get_object(Bucket=dl.bucket_name,
                                        Key=  'ref/annotations/cvat/11_2019-06-20_08-58-01_00001.03601.xml')
sample_xml = result['Body'].read().decode()
sample_xml

'<?xml version="1.0" encoding="utf-8"?>\n<annotations>\n  <version>1.1</version>\n  <meta>\n    <task>\n      <id>11</id>\n      <name>2019-06-20_08-58-01_00001.03601</name>\n      <size>245</size>\n      <mode>interpolation</mode>\n      <overlap>5</overlap>\n      <bugtracker></bugtracker>\n      <created>2019-07-08 18:34:13.981309+03:00</created>\n      <updated>2019-07-08 18:52:45.040892+03:00</updated>\n      <start_frame>0</start_frame>\n      <stop_frame>244</stop_frame>\n      <frame_filter></frame_filter>\n      <labels>\n        <label>\n          <name>vehicle</name>\n          <attributes>\n            <attribute>\n              <name>parked</name>\n              <mutable>True</mutable>\n              <input_type>checkbox</input_type>\n              <default_value>false</default_value>\n              <values>false</values>\n            </attribute>\n            <attribute>\n              <name>stopped</name>\n              <mutable>True</mutable>\n              <input_type>

In [16]:
# dl.download_file('ref/annotations/cvat/11_2019-06-20_08-58-01_00001.03601.xml',
#                  '../../11_2019-06-20_08-58-01_00001.03601.xml')


## Refine parsing code

In [61]:
import xml.etree.ElementTree as ElementTree
from traffic_analysis.d05_evaluation.parse_annotation import parse_annotation

root = ElementTree.fromstring(sample_xml)

# elems = [elem.tag for elem in root.iter()]
# for child in root.iter('start_frame'): 
#     print(child.tag, child.attrib)
#     if child.tag == 'start_frame': 
#         print(child.text)
        


0
244
0


In [65]:
import pandas as pd 

def parse_annotation(xml_root) -> pd.DataFrame:
    """Returns annotation xml file as a pandas dataframe
    """

    annotated_result = {'vehicle_id': [],
                        'frame_id': [],
                        'bboxes': [],
                        'vehicle_type': [],
                        'parked': [],
                        'stopped': [],
                        'start_frame': 0,
                        'stop_frame': 0}

    for task in root.iter('task'):
        for task_info in task.iter():
            if task_info.tag == 'start_frame': 
                annotated_result['start_frame'] = task_info.text
            if task_info.tag == 'stop_frame':
                annotated_result['stop_frame'] = task_info.text
            

    for track in xml_root.iter('track'):
        if track.attrib['label'] == 'vehicle':
            for frame in track.iter('box'):
                annotated_result['vehicle_id'].append(int(track.attrib['id']))
                annotated_result['frame_id'].append(int(frame.attrib['frame']))
                annotated_result['bboxes'].append([float(frame.attrib['xtl']), float(frame.attrib['ytl']),
                                                        float(frame.attrib['xbr']), float(frame.attrib['ybr'])])
                for attr in frame.iter('attribute'):
                    # If name is 'type' then index the dictionary using 'vehicle_type'
                    if attr.attrib['name'] == 'type':
                        annotated_result['vehicle_type'].append(attr.text)
                    # Else just use the name for indexing
                    else:
                        annotated_result[attr.attrib['name']].append(attr.text)

    ground_truth_df = pd.DataFrame.from_dict(annotated_result)
    return ground_truth_df


In [66]:
gt_df = parse_annotation(root)
gt_df

,vehicle_id,frame_id,bboxes,vehicle_type,parked,stopped,start_frame,stop_frame
0,0,40,"[219.48, 83.56, 230.85, 106.93]",motorbike,false,false,0,244
1,0,41,"[220.24, 84.13, 231.61, 107.5]",motorbike,false,false,0,244
2,0,42,"[221.0, 84.7, 232.36, 108.07]",motorbike,false,false,0,244
3,0,43,"[221.75, 85.27, 233.12, 108.64]",motorbike,false,false,0,244
4,0,44,"[222.51, 85.84, 233.88, 109.2]",motorbike,false,false,0,244
5,0,45,"[223.27, 86.4, 234.64, 109.77]",motorbike,false,false,0,244
6,0,46,"[224.03, 86.97, 235.4, 110.34]",motorbike,false,false,0,244
7,0,47,"[224.78, 87.54, 236.15, 110.91]",motorbike,false,false,0,244
8,0,48,"[225.54, 88.11, 236.91, 111.48]",motorbike,false,false,0,244
9,0,49,"[226.3, 88.68, 237.67, 112.05]",motorbike,false,false,0,244


In [13]:
gt_df.sort_values("frame_id")

,vehicle_id,frame_id,bboxes,vehicle_type,parked,stopped
1986,13,3,"[129.16, 52.61, 141.79, 67.14]",car,false,false
1502,11,3,"[136.74, 56.4, 161.37, 83.56]",truck,false,false
851,8,3,"[187.9, 136.62, 252.95, 262.3]",truck,false,false
1744,12,3,"[180.95, 61.46, 203.06, 82.3]",car,false,false
1260,10,3,"[249.17, 140.4, 293.38, 189.67]",car,false,false
1093,9,3,"[215.69, 72.82, 254.22, 130.3]",truck,false,false
1503,11,4,"[136.74, 56.21, 161.21, 83.29]",truck,false,false
1987,13,4,"[129.01, 52.46, 141.57, 66.97]",car,false,false
1094,9,4,"[216.38, 73.43, 254.9, 130.9]",truck,false,false
1745,12,4,"[181.5, 61.7, 203.61, 82.55]",car,false,false
